# Classificação Multiclasse

###### Classificação Multiclasse para prever o resultado de uma partida

Usaremos a Classificação Multiclasse com Árvores de Decisão para construir um modelo capaz de prever o resultado de uma partida de futebol com 3 resultados possíveis: vitória, derrota, empate

In [2]:
import findspark
findspark.init()

In [41]:
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import Row # Pré Processar os dados
from pyspark.ml.feature import StringIndexer
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassficationEvaluator

ImportError: cannot import name 'MulticlassClassficationEvaluator' from 'pyspark.ml.evaluation' (C:\Users\alyss\anaconda3\lib\site-packages\pyspark\ml\evaluation.py)

# Carregando os Dados

In [4]:
sc = SparkContext(appName = 'Lab5')

In [5]:
# Cria a sessão spark
spSession = SparkSession.builder.master("local").getOrCreate()

In [6]:
# Carrega dados gerando um RDD*
dados = sc.textFile("C:/formacao_dataScience_DSA_DADOS/01_bigData_RealTime_Python_Spark/cap14_Apache_Spark_MLlib/Lab5/dados/dataset2.csv")

In [8]:
dados.take(5)

['media_faltas_sofridas,media_faltas_recebidas,media_cartoes_recebidos,media_chutes_a_gol,resultado',
 '4.8,3,1.4,0.3,vitoria',
 '5.1,3.8,1.6,0.2,vitoria',
 '4.6,3.2,1.4,0.2,vitoria',
 '5.3,3.7,1.5,0.2,vitoria']

In [9]:
# Colocando RDD em cache, para otimizar a performance
dados.cache()

C:/formacao_dataScience_DSA_DADOS/01_bigData_RealTime_Python_Spark/cap14_Apache_Spark_MLlib/Lab5/dados/dataset2.csv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [10]:
dados.count()

151

In [11]:
# Removendo a primeira linha do RDD
dados = dados.filter(lambda x: "media_faltas_recebidas" not in x)
dados.count()

150

# Limpeza e transformação dos dados

In [12]:
dados = dados.map(lambda x: x.split(','))

In [25]:
# Mapeamento das colunas 
dados2 = dados.map(lambda p: Row(media_faltas_sofridas = float(p[0]),
                                media_faltas_recebidas = float(p[1]),
                                media_cartoes_recebidos = float(p[2]),
                                media_chutes_a_gol = float(p[3]),
                                resultado = p[4]))

# Cada linha do conjunto de dados será convertido para o tipo linha (Row)
# Isso é importante para atribuirmos Schemas às colunas, pois os dados
# brutos no formato RDD não possuem Schema

dados2.take(5)

[Row(media_faltas_sofridas=4.8, media_faltas_recebidas=3.0, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.3, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.8, media_cartoes_recebidos=1.6, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=4.6, media_faltas_recebidas=3.2, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.3, media_faltas_recebidas=3.7, media_cartoes_recebidos=1.5, media_chutes_a_gol=0.2, resultado='vitoria'),
 Row(media_faltas_sofridas=5.1, media_faltas_recebidas=3.5, media_cartoes_recebidos=1.4, media_chutes_a_gol=0.2, resultado='vitoria')]

In [21]:
# Converte o RDD para DataFrame do Spark
df_time = spSession.createDataFrame(dados2)

In [22]:
df_time.show(5)

+---------------------+----------------------+-----------------------+------------------+---------+
|media_faltas_sofridas|media_faltas_recebidas|media_cartoes_recebidos|media_chutes_a_gol|resultado|
+---------------------+----------------------+-----------------------+------------------+---------+
|                  4.8|                   3.0|                    1.4|               0.3|  vitoria|
|                  5.1|                   3.8|                    1.6|               0.2|  vitoria|
|                  4.6|                   3.2|                    1.4|               0.2|  vitoria|
|                  5.3|                   3.7|                    1.5|               0.2|  vitoria|
|                  5.1|                   3.5|                    1.4|               0.2|  vitoria|
+---------------------+----------------------+-----------------------+------------------+---------+
only showing top 5 rows



In [23]:
# Criando um indice numerico para a coluna label target
stringIndexer = StringIndexer(inputCol = 'resultado', outputCol = 'idx_resultado')


In [26]:
# Treinando o String Indexer
si_model= stringIndexer.fit(df_time)

In [27]:
# Aplicando o string indexer
df_time_final = si_model.transform(df_time)

In [28]:
df_time_final.select('resultado', 'idx_resultado').distinct().collect()

[Row(resultado='derrota', idx_resultado=0.0),
 Row(resultado='vitoria', idx_resultado=2.0),
 Row(resultado='empate', idx_resultado=1.0)]

# Análise Exploratória

In [29]:
# Estatística descritiva
df_time_final.describe().show()

+-------+---------------------+----------------------+-----------------------+------------------+---------+------------------+
|summary|media_faltas_sofridas|media_faltas_recebidas|media_cartoes_recebidos|media_chutes_a_gol|resultado|     idx_resultado|
+-------+---------------------+----------------------+-----------------------+------------------+---------+------------------+
|  count|                  150|                   150|                    150|               150|      150|               150|
|   mean|    5.843333333333332|    3.0573333333333337|      3.758000000000001|1.1993333333333331|     null|               1.0|
| stddev|   0.8280661279778625|   0.43586628493669793|     1.7652982332594667|0.7622376689603465|     null|0.8192319205190404|
|    min|                  4.3|                   2.0|                    1.0|               0.1|  derrota|               0.0|
|    max|                  7.9|                   4.4|                    6.9|               2.5|  vitoria|    

In [32]:
for i in df_time_final.columns:
    # Verificando se a coluna é do tipo string
    if not(isinstance(df_time_final.select(i).take(1)[0][0], str)):
        print(f"Correlação da Variável Target com {i}: {df_time_final.stat.corr('idx_resultado', i)}")

Correlação da Variável Target com media_faltas_sofridas: -0.4600391565002369
Correlação da Variável Target com media_faltas_recebidas: 0.6183715308237437
Correlação da Variável Target com media_cartoes_recebidos: -0.649241830764174
Correlação da Variável Target com media_chutes_a_gol: -0.5803770334306265
Correlação da Variável Target com idx_resultado: 1.0


# Pré Processamento dos Dados

In [33]:
# Criando um LabeledPoint (targer, Vector(features))
# Remove colunas não relevantes para o modelo ou com baixa correlação
def transformaVar(row):
    obj = (row['resultado'], row['idx_resultado'],
          Vectors.dense([row['media_faltas_sofridas'],
                        row['media_faltas_recebidas'],
                        row['media_cartoes_recebidos'],
                        row['media_chutes_a_gol']]))
    return obj

In [34]:
# Aplica a função em novo RDD
df_time_final_RDD = df_time_final.rdd.map(transformaVar)

In [35]:
df_time_final_RDD.take(5)

[('vitoria', 2.0, DenseVector([4.8, 3.0, 1.4, 0.3])),
 ('vitoria', 2.0, DenseVector([5.1, 3.8, 1.6, 0.2])),
 ('vitoria', 2.0, DenseVector([4.6, 3.2, 1.4, 0.2])),
 ('vitoria', 2.0, DenseVector([5.3, 3.7, 1.5, 0.2])),
 ('vitoria', 2.0, DenseVector([5.1, 3.5, 1.4, 0.2]))]

In [49]:
# Converte novamente para dataFrame do Spark
df_spark2 = spSession.createDataFrame(df_time_final_RDD['resultado', 'label', 'features'])

TypeError: 'PipelinedRDD' object is not subscriptable

In [52]:
df_spark = df_time_final_RDD.toDF()

In [ ]:
df_spark = df_spark.withColumnRenamed("

In [ ]:
resultado

In [59]:
df_spark = df_spark.withColumnRenamed("_1", "resultado")

In [60]:
df_spark = df_spark.withColumnRenamed("_2", "label")

In [61]:
df_spark = df_spark.withColumnRenamed("_3", "features")

In [62]:
df_spark.show(10)

+---------+-----+-----------------+
|resultado|label|         features|
+---------+-----+-----------------+
|  vitoria|  2.0|[4.8,3.0,1.4,0.3]|
|  vitoria|  2.0|[5.1,3.8,1.6,0.2]|
|  vitoria|  2.0|[4.6,3.2,1.4,0.2]|
|  vitoria|  2.0|[5.3,3.7,1.5,0.2]|
|  vitoria|  2.0|[5.1,3.5,1.4,0.2]|
|  vitoria|  2.0|[4.9,3.0,1.4,0.2]|
|  vitoria|  2.0|[4.7,3.2,1.3,0.2]|
|  vitoria|  2.0|[4.6,3.1,1.5,0.2]|
|  vitoria|  2.0|[5.0,3.6,1.4,0.2]|
|  vitoria|  2.0|[5.4,3.9,1.7,0.4]|
+---------+-----+-----------------+
only showing top 10 rows



In [63]:
df_spark.cache()

DataFrame[resultado: string, label: double, features: vector]

In [64]:
# Divisão dos dados em treino e teste
(dados_treino, dados_teste) = df_spark.randomSplit([0.7, 0.3])

In [65]:
dados_treino.count()

107

In [66]:
dados_teste.count()

43